In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
from typing import Tuple

def carregar_dados(filename: str) -> pd.DataFrame:
    dados = pd.read_csv(filename, sep=";", encoding="latin-1")
    dados['Data do pedido'] = pd.to_datetime(dados['Data do pedido'], errors='coerce')
    dados['Data de envio'] = pd.to_datetime(dados['Data de envio'], errors='coerce')
    dados['Quantidade'] = dados['Quantidade'].astype(int)
    dados['Vendas'] = pd.to_numeric(dados['Vendas'].str.replace(',', '.'))
    return dados

def tratamento_outliers(data: pd.Series, threshold: float = 3.0) -> pd.Series:
    mean = data.mean()
    std = data.std()
    is_outlier = (data - mean).abs() > threshold * std
    data_copy = data.copy()
    data_copy[is_outlier] = data.median()
    return data_copy

def dividir_dados(dados: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    dados['Vendas'] = tratamento_outliers(dados['Vendas'])
    X = dados.index.astype('int64').values.reshape(-1, 1)
    y = dados['Vendas'].values
    return X, y

def treinar_modelo(X: np.ndarray, y: np.ndarray) -> XGBRegressor:
    tscv = TimeSeriesSplit(n_splits=5)
    
    param_grid = {
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [5, 10, 15]
    }
    
    best_model = None
    best_mse = float('inf')
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        grid_search = GridSearchCV(XGBRegressor(objective='reg:squarederror', random_state=42), param_grid,
                                   cv=3, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
    
        previsoes = model.predict(X_test)
    
        mse = mean_squared_error(y_test, previsoes) 
        print(f'MSE nos conjuntos de teste: {mse:.2f}')
    
        if mse < best_mse:
            best_mse = mse
            best_model = model
    
    return best_model

def criar_grafico(dados: pd.DataFrame, modelo: XGBRegressor) -> None:
    X, y = dividir_dados(dados)
    modelo.fit(X, y)
    previsoes = modelo.predict(X)
    df_resultados = pd.DataFrame({'Data': dados.index, 'Vendas_Real': y, 'Vendas_Previstas': previsoes})
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_resultados['Data'], y=df_resultados['Vendas_Real'],
                             mode='lines', name='Vendas Reais', line=dict(color='blue')))
    
    fig.add_trace(go.Scatter(x=df_resultados['Data'], y=df_resultados['Vendas_Previstas'],
                             mode='lines', name='Previsões', line=dict(color='red')))
    
    fig.update_layout(
        title='Vendas - Previsao | Valor Real',
        xaxis_title='Data do Pedido',
        yaxis_title='Vendas',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white', size=18),
        showlegend=True,
        hovermode='x unified'
    )
    
    fig.update_layout(legend=dict(font=dict(size=16)))
    fig.update_layout(autosize=True)
    fig.show()
    
    for index, row in df_resultados.iterrows():
        data = row['Data']
        real = float(row['Vendas_Real'])
        previsto = float(row['Vendas_Previstas'])
        print(f'Data: {data}, Valor Real: {real:.2f}, Valor Previsto: {previsto:.2f}')

if __name__ == "__main__":
    filename = r'D:\seriestesteapp\super_loja.csv'
    dados = carregar_dados(filename)
    X, y = dividir_dados(dados)
    modelo = treinar_modelo(X, y) 
    criar_grafico(dados, modelo)



MSE nos conjuntos de teste: 6116818.32
MSE nos conjuntos de teste: 5649099.21
MSE nos conjuntos de teste: 5657180.53
MSE nos conjuntos de teste: 5993555.15
MSE nos conjuntos de teste: 4025274.18


Data: 0.0, Valor Real: 130.80, Valor Previsto: 1323.45
Data: 1.0, Valor Real: 2521.60, Valor Previsto: 1323.45
Data: 2.0, Valor Real: 1932.80, Valor Previsto: 1376.36
Data: 3.0, Valor Real: 354.40, Valor Previsto: 1218.27
Data: 4.0, Valor Real: 716.00, Valor Previsto: 1218.27
Data: 5.0, Valor Real: 561.20, Valor Previsto: 1218.27
Data: 6.0, Valor Real: 561.20, Valor Previsto: 1218.27
Data: 7.0, Valor Real: 3446.40, Valor Previsto: 1973.30
Data: 8.0, Valor Real: 973.60, Valor Previsto: 1922.24
Data: 9.0, Valor Real: 3415.20, Valor Previsto: 1939.15
Data: 10.0, Valor Real: 120.60, Valor Previsto: 1095.42
Data: 11.0, Valor Real: 207.60, Valor Previsto: 1095.42
Data: 12.0, Valor Real: 2106.40, Valor Previsto: 1233.28
Data: 13.0, Valor Real: 801.00, Valor Previsto: 1233.28
Data: 14.0, Valor Real: 1326.40, Valor Previsto: 1233.28
Data: 15.0, Valor Real: 129.40, Valor Previsto: 1233.28
Data: 16.0, Valor Real: 188.40, Valor Previsto: 1233.28
Data: 17.0, Valor Real: 3082.80, Valor Previsto: 152